# Introducción al Aprendizaje Automático: Taller 2



## Predicción de Riesgo Crediticio

El objetivo de este taller es que se familiciaricen con competencias/desafíos en ciencia de datos y machine learning. Por lo tanto, la actividad es abierta y requiere diseñar, implementar, y evaluar una solución a un problema real usando técnicas estudiadas en el curso.


<table align='left'>
<tr>
<td><img src='https://content.pymnts.com/wp-content/uploads/2019/02/Aire-AI-Credit-Scoring-1000x600.jpg' width='900'/> </td>
</tr>
</table>

## Introducción

La tecnología financiera (*fintech*) es un conjunto de productos y servicios que aprovechan avances tecnOlógicos recientes para resolver problemas o mejorar procesos actuales en la industria financiera. Este sector representa una de las áreas laborales de mayor crecimiento reciente.

El aprendizaje automático en fintech implica la aplicación de algoritmos y modelos avanzados para analizar datos financieros, hacer predicciones, automatizar procesos y optimizar la toma de decisiones en la industria financiera. Permite a las instituciones financieras mejorar la evaluación de riesgos, la detección de fraudes, el servicio al cliente, las estrategias de inversión y más, aprovechando el poder de los datos y la automatización.

## Problema & Datos

> **Problema:** En este desafío exploraremos el problema de predecir si una solicitud de crédito a un banco será pagada o no, una versión binaria de lo que se denomina análisis de riesgo, que es su vez una de las aplicaciones más comunes del aprendizaje automático en la industria financiera.

> **Materiales:** Usted dispondrá de varias tablas extraídas de una base de datos relacional que describe el comportamiento previo de los clientes en una entidad financiera real. La solución del desafío requiere emplear técnicas de fusión de datos para crear un dataset que pueda ser utilizado por modelos clásicos de aprendizaje automático. Como en muchos problemas reales, la construcción de una solución requerirá también de una limpieza de datos, de un método para tratar datos faltantes, de un buen análisis exploratorio de datos, y de una creativa ingeniería de atributos.

## Métrica

La métrica de la competencia será [AUC](https://es.wikipedia.org/wiki/Curva_ROC) ó área bajo la curva ROC.

$$\operatorname{TPR}(T): T \to y(x)$$
$$\operatorname{FPR}(T): T \to x$$

\begin{aligned}
A & = \int_{x=0}^{1}{\mbox{TPR}(\mbox{FPR}^{-1}(x)) dx}\\
& = \int_{\infty}^{-\infty} \mbox{TPR}(T) \mbox{FPR}'(T) \, dT\\
& = \int_{-\infty}^\infty \int_{-\infty}^\infty I(T' \ge T)f_1(T') f_0(T) \, dT' \, dT = P(X_1 \ge X_0)
\end{aligned}



## Base Imports

In [4]:
# Plotting libraries
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
# Scikit learn
import sklearn.preprocessing as skp
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
# Scikit Metrics
from sklearn.metrics import log_loss, classification_report, accuracy_score, f1_score,roc_auc_score
# Data manipulation libraries
import pandas as pd
import numpy as np
# General use
from tqdm.notebook import tqdm
from joblib import dump, load
import warnings
warnings.filterwarnings("ignore")


# Si quiere trabajar en local cambiar a False
COLAB_FLAG = False

## Carga de Datos

Como es usual, debe cargar los archivos distribuidos con esta tarea en la máquina virtual o en la máquina local donde trabajará, sustituyendo las rutas mostradas como ejemplo por las rutas donde tiene alojados los archivos.

In [5]:
data_path = '../Data/'

if COLAB_FLAG:
  print("Mounting your Google Drive ...")

  from google.colab import drive
  drive.flush_and_unmount()
  drive.mount('/content/drive', force_remount=True)

  data_path = "/content/drive/My Drive/MA_datasets/Ejemplos/" # Esta es la ruta. Recuerde modificarla acorde a sus necesidades

## Unzip Data Files

In [6]:
#!unzip '/content/drive/My Drive/MA_datasets/Ejemplos/*.zip' -d '/content/drive/My Drive/MA_datasets/Ejemplos/'

Movemos y renombramos la carpeta a la ubicación que queramos

In [7]:
#!mv content/file_data/ train_data/

## Funciones útiles

Estas funciones son adheridas al notebook con tal de ayudar en la inspiración de visualizaciones útiles, facilitando así el encontrar patrones para unificar las tablas.

In [8]:
def numeric_columns(df):
  numeric_columns = []
  for column in df.columns.tolist():
    if df.dtypes[column] != 'object':
      numeric_columns.append(column)
  return numeric_columns

def plot_numeric_data(df, nrows=3, ncols=4, exclude_columns=['SK_ID_BUREAU', 'SK_ID_CURR']):
  fig, ax = plt.subplots(figsize=(16, 18), nrows=nrows, ncols=ncols)
  cont = 0
  for idx, column in enumerate(tqdm(numeric_columns(df))):
    if df.dtypes[column] != 'object' and column not in exclude_columns:
      df[column].plot(kind='kde',
                      xlabel=column,
                      ax=ax[(idx + cont)//4, (idx + cont)%4])
      ax[(idx + cont)//4, (idx + cont)%4].set_xlabel(column, fontsize=8)
    else:
      cont -= 1

  plt.subplots_adjust(bottom=.25, left=.25)
  plt.tight_layout()
  plt.show()

# Descripción de las columnas
def dataframe_columns_description(df):
  print('-' * 160)
  for index, row in description_df[description_df['Row'].isin(df)].drop('Table', axis=1).iterrows():
    print(f"|{row['Row']}: {row['Description']} ({row['Special']})" if type(row['Special']) != float else f"|{row['Row']}: {row['Description']}")
    print('-' * 160)

## Problema & Datos

In [9]:
description_df = pd.read_csv(data_path + 'columns_description.csv', index_col=0)
loan_history_df = pd.read_csv(data_path + 'historial_prestamos.csv', index_col=0).drop(['Unnamed: 0'], axis=1)
loan_history_movements_df = pd.read_csv(data_path + 'historial_prestamos_movimientos.csv', index_col=0).drop(['Unnamed: 0'], axis=1)
request_test_df = pd.read_csv(data_path + 'solicitudes_test.csv', index_col=0)
request_train_df = pd.read_csv(data_path + 'solicitudes_train.csv', index_col=0)
credit_card_movements_df = pd.read_csv(data_path + 'tarjeta_credito_movimientos.csv', index_col=0).drop(['Unnamed: 0'], axis=1)

KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
loan_history_df.to_csv('historial_prestamos.csv', index=False)
loan_history_movements_df.to_csv('historial_prestamos_movimientos.csv', index=False)
credit_card_movements_df.to_csv('tarjeta_credito_movimientos.csv', index=False)
request_train_df.to_csv('solicitudes_train.csv', index=False)
request_test_df.to_csv('solicitudes_test.csv', index=False)

In [ ]:
display(description_df.head())

,Table,Row,Description,Special
0,solicitud_{train/test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,solicitud_{train/test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,solicitud_{train/test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,solicitud_{train/test}.csv,CODE_GENDER,Gender of the client,NaN
4,solicitud_{train/test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN


In [ ]:
display(loan_history_df.head())
display(loan_history_movements_df.head())

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [ ]:
display(credit_card_movements_df.head())

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2582071,363914,-1,63975.555,45000,2250.0,2250.000,0.0,0.000,2250.00,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
1,1389973,337855,-4,236572.110,225000,2250.0,2250.000,0.0,0.000,11795.76,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
2,1891521,126868,-1,453919.455,450000,0.0,11547.000,0.0,11547.000,22924.89,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0
3,2181852,367360,-4,291543.075,292500,90000.0,289339.425,0.0,199339.425,130.50,...,286831.575,286831.575,3.0,8,0.0,5.0,3.0,Active,0,0
4,1235299,203885,-5,201261.195,225000,76500.0,111026.700,0.0,34526.700,6338.34,...,197224.695,197224.695,3.0,9,0.0,6.0,38.0,Active,0,0


In [ ]:
display(request_train_df.head())

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,331475,0,Cash loans,F,Y,Y,0,135000.0,835380.0,30955.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,366488,0,Cash loans,F,N,Y,0,103500.0,659533.5,26284.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
2,444791,0,Cash loans,F,N,Y,0,135000.0,219042.0,23130.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
3,112960,0,Revolving loans,F,N,Y,0,67500.0,675000.0,33750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,9.0
4,418735,0,Revolving loans,F,N,Y,0,315000.0,900000.0,45000.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0


Como sugerencia al con tablas con muchos *NaNs* o también denominada *Raw Data* una opción es prescindir de aquellas columnas que posean una cantidad de *NaNs* superior a cierto *treshhold* por sobre el cual pierde sentido la utilización del dato.

A modo de ejemplo a continuación filtramos el dataset, dejando fuera toda las columnas que tengan *NaNs*.

Este filtrado debe ser realizado con especial cuidado pues se puede incurrir en la pérdida de información valiosa donde el valor perdido puede ser determinado a partir de las otras columnas o incluso las otras tablas.


In [ ]:
nans_cols = []
for idx, row in pd.DataFrame(request_train_df.isna().sum()).iterrows():
  if row[0] == 0:
    nans_cols.append(idx)

# Importante la utilización de copy para que python explicite un nuevo espacio de memoria para nuevo dataframe
request_train_filtered_df = request_train_df[nans_cols].copy()
request_train_filtered_df

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,NAME_INCOME_TYPE,...,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21
0,331475,0,Cash loans,F,Y,Y,0,135000.0,835380.0,Pensioner,...,0,0,0,0,0,0,0,0,0,0
1,366488,0,Cash loans,F,N,Y,0,103500.0,659533.5,Pensioner,...,0,0,0,0,0,0,0,0,0,0
2,444791,0,Cash loans,F,N,Y,0,135000.0,219042.0,Working,...,0,0,0,0,0,0,0,0,0,0
3,112960,0,Revolving loans,F,N,Y,0,67500.0,675000.0,Working,...,0,0,0,0,0,0,0,0,0,0
4,418735,0,Revolving loans,F,N,Y,0,315000.0,900000.0,Pensioner,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150674,244356,0,Cash loans,M,N,Y,0,112500.0,112068.0,Working,...,0,0,0,0,0,0,0,0,0,0
150675,213955,0,Cash loans,F,N,Y,0,135000.0,781920.0,State servant,...,0,0,0,0,0,0,0,0,0,0
150676,376877,0,Revolving loans,F,N,Y,0,135000.0,202500.0,Pensioner,...,0,0,0,0,0,0,0,0,0,0
150677,246747,0,Cash loans,F,N,N,1,135000.0,508495.5,Working,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# utilización del descriptor de columnas
print('\nloan_history_df:\n')
dataframe_columns_description(loan_history_df)
print('\nloan_history_movements_df:\n')
dataframe_columns_description(loan_history_movements_df)


loan_history_df:

----------------------------------------------------------------------------------------------------------------------------------------------------------------
|SK_ID_CURR: ID of loan in our sample
----------------------------------------------------------------------------------------------------------------------------------------------------------------
|AMT_ANNUITY: Loan annuity
----------------------------------------------------------------------------------------------------------------------------------------------------------------
|SK_ID_CURR: ID of loan in our sample - one loan in our sample can have 0,1,2 or more related previous credits in credit bureau  (hashed)
----------------------------------------------------------------------------------------------------------------------------------------------------------------
|CREDIT_ACTIVE: Status of the Credit Bureau (CB) reported credits
--------------------------------------------------------------------

Las siguientes secciones de código son dejadas con tal de inspirarlos a utilizar el contexto relacionado al cliente. Al igual que para ilustrar algunas de las operaciones que pueden realizar para unificar los *dataframes*.
Queda a criterio del estudiante la utilización de estos códigos para enriquecer o no los datos.


In [ ]:
print(f"Percentage of Closed Loan credits: {len(loan_history_df[loan_history_df['CREDIT_ACTIVE'] == 'Closed']) / len(loan_history_df)}")
print(f"Percentage of Active Loan credits: {len(loan_history_df[loan_history_df['CREDIT_ACTIVE'] == 'Active']) / len(loan_history_df)}")
print(f'Forma del dataframe {loan_history_df.shape}')
loan_history_df.head()

Percentage of Closed Loan credits: 0.6257976641177164
Percentage of Active Loan credits: 0.3703715981362081
Forma del dataframe (1025651, 17)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [ ]:
plot_numeric_data(loan_history_df[loan_history_df['CREDIT_ACTIVE'] == 'Active'].groupby(['SK_ID_CURR']).mean())

In [ ]:
# Obtenemos las filas del último movimiento
loan_history_movements_df.loc[loan_history_movements_df.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].idxmin()]

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
1369403,5008804,-15,X
1369418,5008805,-14,X
1684675,5008806,-29,X
1684683,5008807,-87,0
14023444,5008808,-4,X
...,...,...,...
12581113,6842884,-47,X
12581137,6842885,-23,0
12581170,6842886,-32,0
12581207,6842887,-36,0


In [ ]:
print(f'loan_history_df: {len(loan_history_df)}\nloan_history_movements_df: {len(loan_history_movements_df)}')
print(len(loan_history_movements_df['SK_ID_BUREAU'].unique()))

test_df = loan_history_df.merge(loan_history_movements_df, how='right', on='SK_ID_BUREAU') #.head()
test_df.loc[test_df.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].idxmin()].head()

loan_history_df: 1025651
loan_history_movements_df: 10314567
366508


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,MONTHS_BALANCE,STATUS
964037,163686,5008804,Active,currency 1,-477,0,984.0,NaN,88363.845,0,2556900.0,657766.53,0.0,0.0,Car loan,-9,NaN,-15,X
964052,163686,5008805,Closed,currency 1,-425,0,-233.0,-233.0,18261.585,0,675000.0,0.00,0.0,0.0,Consumer credit,-233,29118.195,-14,X
1190184,356203,5008806,Closed,currency 1,-904,0,435.0,-216.0,NaN,0,175500.0,0.00,0.0,0.0,Consumer credit,-115,0.000,-29,X
1190192,356203,5008807,Closed,currency 1,-2667,0,-840.0,-1245.0,NaN,0,562500.0,0.00,0.0,0.0,Consumer credit,-181,5690.520,-87,0
9834485,260295,5008808,Active,currency 1,-133,0,1696.0,NaN,NaN,0,135000.0,0.00,135000.0,0.0,Credit card,-69,0.000,-4,X
